### 절대 모멘텀
1. 파생변수 'STD-YM' 생성 -> 인덱스에서 년-월을 추출하여 대입 
2. 'STD-YM' 별 마지막날의 데이터들을 month_last_df 추가 (단순 행 결합)
3. 전월의 수정 종가 값을 가진 파생변수 생성 
4. 전년도의 수정 종가 값을 가진 파생변수 생성 
5. 전월의 데이터와 전년도의 데이터를 이용하여 거래 내역 생성 
6. 수익율 계산

In [22]:
import pandas as pd
import numpy as np 
from datetime import datetime
import warnings

In [ ]:
# 위험 메시지 출력 필터
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../../csv/AMZN.csv")

In [3]:
if 'Date' in df.columns:
    df.set_index('Date', inplace=True)

In [4]:
# index를 시계열로 전환
df.index = pd.to_datetime(df.index)

In [11]:
# df에 STD-YM 컬럼을 생성하여 인덱스에서 년도-월 데이터를 추출하여 대입
df['STD-YM'] = df.index.strftime('%Y-%m')

In [ ]:
# 빈 데이터프레임에 월말의 데이터들을 행결합 
# STD-YM에서 다음 행과 현재 행이 다르다면
flag = df['STD-YM'] != df['STD-YM'].shift(-1)
df.loc[flag, ]

In [ ]:
# 기준년월의 유니크 값을 따로 생성 
month_list = df['STD-YM'].unique()
month_list

In [17]:
month_last_df = pd.DataFrame()

for month in month_list:
    # flag = df['STD-YM'] == month
    # data = df.loc[flag, ].tail(1)
    data = df.loc[month, ].tail(1)
    month_last_df = pd.concat(
        [month_last_df, data], axis=0
    )

In [20]:
month_last_df = month_last_df[['Adj Close', 'STD-YM']]
month_last_df.head()

,Adj Close,STD-YM
Date,,
1997-05-30,1.500000,1997-05
1997-06-30,1.541667,1997-06
1997-07-31,2.395833,1997-07
1997-08-29,2.338542,1997-08
1997-09-30,4.338542,1997-09


In [26]:
# 전월의 수정종가 파생변수(BF-1M) 생성
month_last_df['BF-1M'] = month_last_df['Adj Close'].shift(1).fillna(0)
# 전년도의 수정종가 파생변수(BF-12M) 생성
month_last_df['BF-12M'] = month_last_df['Adj Close'].shift(12).fillna(0)

In [30]:
month_last_df

,Adj Close,STD-YM,BF-1M,BF-12M
Date,,,,
1997-05-30,1.500000,1997-05,0.000000,0.000000
1997-06-30,1.541667,1997-06,1.500000,0.000000
1997-07-31,2.395833,1997-07,1.541667,0.000000
1997-08-29,2.338542,1997-08,2.395833,0.000000
1997-09-30,4.338542,1997-09,2.338542,0.000000
...,...,...,...,...
2019-02-28,1639.829956,2019-02,1718.729980,1512.449951
2019-03-29,1780.750000,2019-03,1639.829956,1447.339966
2019-04-30,1926.520020,2019-04,1780.750000,1566.130005


- 거래 내역 추가 
    - (전월의 수정주가) / (전년도의 수정주가) - 1 값이 0보다 크고 무한대가 아닌 경우가 매수 타이밍
    - 모멘텀 인덱스가 위의 조건에 부합할 경우 df의 trade 컬럼에 해당 월에 구매 내역 추가 

In [ ]:
df['trade'] = ""

# month_last_df의 인덱스를 기준으로 반복문 실행 
for idx in month_last_df.index:
    signal = ""

    # 절대 모멘텀 인덱스를 계산 
    momentum_index = (month_last_df.loc[idx, 'BF-1M'] / 
                    month_last_df.loc[idx, 'BF-12M']) - 1
    # print(momentum_index)
    # break
    # 조건식 : 모멘텀 인덱스가 0보다 크고 무한대가 아닌
    # 참인 경우 데이터 if 조건식 else 거짓인 경우 데이터
    flag = True if (momentum_index > 0) & (momentum_index != np.inf) \
        else False

    if flag:
        signal = 'buy'
    print(f"날짜 : {idx}, 모멘텀 인덱스 : {momentum_index}, signal : {signal}")
    df.loc[idx:, 'trade'] = signal

In [29]:
df['trade'].value_counts()

trade
buy    4033
       1530
Name: count, dtype: int64